In [1]:
import re
import os, glob, datetime, time
import numpy as np
import torch
import torch.nn as nn
from torch.nn.modules.loss import _Loss
import torch.nn.init as init
from torch.utils.data import DataLoader
import torch.optim as optim

from skimage import io, color
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr
import matplotlib.pyplot as plt

# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
gamma = 0.5
sigmaU = 1
sigmaW = 10
epochs = 100

model_name = 'SDN_Color_Block1_gamma_%.1f_sigmaU_%.1f_sigmaW_%d'%(gamma, sigmaU, sigmaW)
save_dir = os.path.join('Models', model_name)

In [3]:
class Block1(nn.Module):
    def __init__(self, ch, kernel_size=3, M=6):
        super(Block1, self).__init__()
        layers = []
        
        for i in range(M):
            layers.append(nn.Conv2d(ch, ch, kernel_size, padding=1, bias=False))
            layers.append(nn.BatchNorm2d(ch))
            layers.append(nn.ReLU(inplace=True))
 
        self.conv = nn.Sequential(*layers)

    def forward(self, input):
        return self.conv(input)

class SDNCNN(nn.Module):
    
    def __init__(self, filters=64, image_channels=3, use_bnorm=True, kernel_size=3):
        super(SDNCNN, self).__init__()
        kernel_size = 3
        padding = 1
        self.conv0 = nn.Conv2d(in_channels=image_channels, out_channels=filters, kernel_size=kernel_size, padding=padding)
        
        self.convOut1 = nn.Conv2d(in_channels=filters, out_channels=image_channels, kernel_size=kernel_size, padding=padding, bias=True)
        self.convOut2 = nn.Conv2d(in_channels=filters, out_channels=image_channels, kernel_size=kernel_size, padding=padding, bias=True)
        self.convOut3 = nn.Conv2d(in_channels=filters, out_channels=image_channels, kernel_size=kernel_size, padding=padding, bias=True)
        
        self.ResBlock1 = Block1(filters, M=6)
        self.ResBlock2 = Block1(filters, M=6)
        self.ResBlock3 = Block1(filters, M=6)
        self.ResBlock4 = Block1(filters, M=6)
        self.ResBlock5 = Block1(filters, M=6)
        self.ResBlock6 = Block1(filters, M=6)
        self.ResBlock7 = Block1(filters, M=6)
        
        self._initialize_weights()

    def forward(self, x):

        c0 = self.conv0(x)
        
        c1 = self.ResBlock1(c0)
        
        c2 = self.ResBlock2(c1)
        
        c3 = self.ResBlock3(c2)
        
        c4 = self.ResBlock4(c3)

        c5 = self.ResBlock5(c4+c3)
        c55 = self.convOut1(c5)
        
        c6 = self.ResBlock6(c5+c2)
        c66 = self.convOut2(c6)
        
        c7 = self.ResBlock7(c6+c1)
        c77 = self.convOut3(c7)
        
        noise = c55 + c66 + c77
        rec = x - noise
        
        return rec, noise

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

def findLastCheckpoint(save_dir):
    file_list = glob.glob(os.path.join(save_dir, 'model_*.pth'))
    if file_list:
        epochs_exist = []
        for file_ in file_list:
            result = re.findall(".*model_(.*).pth.*", file_)
            epochs_exist.append(int(result[0]))
        initial_epoch = max(epochs_exist)
    else:
        initial_epoch = 0
    return initial_epoch

In [4]:
model = SDNCNN()
initial_epoch = findLastCheckpoint(save_dir=save_dir)  # load the last model in matconvnet style
if initial_epoch > 0:
    print('resuming by loading epoch %03d' % initial_epoch)
    model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % initial_epoch))

resuming by loading epoch 096


/home/zhy/anaconda3/lib/python3.7/site-packages/torch/serialization.py:400: UserWarning: Couldn't retrieve source code for container of type SDNCNN. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "
/home/zhy/anaconda3/lib/python3.7/site-packages/torch/serialization.py:400: UserWarning: Couldn't retrieve source code for container of type Block1. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


In [8]:
with torch.no_grad():
    np.random.seed(42)
    torch.cuda.manual_seed(42)
    torch.manual_seed(42)
    
    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'CBSD68')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    test_data = './Test_datasets/CBSD68/*.png'
    test_dir = glob.glob(test_data)
    
    initial_epoch = findLastCheckpoint(save_dir=save_dir)
    for e in range(95, 101):

        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(test_dir)):

            start_time = time.time()

            batch_x = io.imread(test_dir[i])
            u = np.random.normal(0, sigmaU, batch_x.shape)
            w = np.random.normal(0, sigmaW, batch_x.shape)
            noise = np.power(batch_x, gamma)*u + w
            batch_y = batch_x + noise

            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)

            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0

            psnr_list += [compare_psnr(out, batch_x)]
            
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()

        print('Epoch:%d, PSNR: %.3f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Epoch:95, PSNR: 33.988, SSIM: 0.9644
Epoch:96, PSNR: 33.991, SSIM: 0.9645
Epoch:97, PSNR: 33.976, SSIM: 0.9643
Epoch:98, PSNR: 34.001, SSIM: 0.9646
Epoch:99, PSNR: 34.002, SSIM: 0.9646
Epoch:100, PSNR: 33.987, SSIM: 0.9645


In [5]:
with torch.no_grad():
    np.random.seed(42)
    torch.cuda.manual_seed(42)
    torch.manual_seed(42)
    
    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'kodak')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    test_data = './Test_datasets/kodak/*.png'
    test_dir = glob.glob(test_data)
    
    initial_epoch = findLastCheckpoint(save_dir=save_dir)
    for e in range(1, initial_epoch+1):

        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(test_dir)):
            start_time = time.time()

            batch_x = io.imread(test_dir[i])
            u = np.random.normal(0, sigmaU, batch_x.shape)
            w = np.random.normal(0, sigmaW, batch_x.shape)
            noise = np.power(batch_x, gamma)*u + w
            batch_y = batch_x + noise
            
            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)
            
            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0

            psnr_list += [compare_psnr(out, batch_x)]
            
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()

        print('Epoch:%d, PSNR: %.3f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Epoch:1, PSNR: 32.406, SSIM: 0.9379
Epoch:2, PSNR: 32.444, SSIM: 0.9385
Epoch:3, PSNR: 32.468, SSIM: 0.9388
Epoch:4, PSNR: 32.443, SSIM: 0.9384
Epoch:5, PSNR: 32.449, SSIM: 0.9385
Epoch:6, PSNR: 33.325, SSIM: 0.9485
Epoch:7, PSNR: 33.320, SSIM: 0.9476
Epoch:8, PSNR: 34.159, SSIM: 0.9582
Epoch:9, PSNR: 34.321, SSIM: 0.9603
Epoch:10, PSNR: 34.337, SSIM: 0.9604
Epoch:11, PSNR: 34.337, SSIM: 0.9604
Epoch:12, PSNR: 34.293, SSIM: 0.9598
Epoch:13, PSNR: 34.296, SSIM: 0.9598
Epoch:14, PSNR: 34.321, SSIM: 0.9602
Epoch:15, PSNR: 34.309, SSIM: 0.9600
Epoch:16, PSNR: 34.300, SSIM: 0.9601
Epoch:17, PSNR: 34.331, SSIM: 0.9603
Epoch:18, PSNR: 34.277, SSIM: 0.9595
Epoch:19, PSNR: 34.297, SSIM: 0.9599
Epoch:20, PSNR: 34.309, SSIM: 0.9600
Epoch:21, PSNR: 34.301, SSIM: 0.9601
Epoch:22, PSNR: 34.332, SSIM: 0.9604
Epoch:23, PSNR: 34.291, SSIM: 0.9598
Epoch:24, PSNR: 33.647, SSIM: 0.9520
Epoch:25, PSNR: 34.355, SSIM: 0.9604
Epoch:26, PSNR: 34.373, SSIM: 0.9606
Epoch:27, PSNR: 34.360, SSIM: 0.9605
Epoch:28, 

In [9]:
with torch.no_grad():

    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'Nam')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    test_dir = './Test_datasets/Real_ccnoise_denoised_part'
    clean_dir = glob.glob(os.path.join(test_dir, '*mean.png'))
    
    result_dir = './Results/Nam15_gamma_%.1f_sigmaU_%.1f_sigmaW_%d'%(gamma, sigmaU, sigmaW)
    if not os.path.exists(result_dir):
        os.mkdir(result_dir)

    initial_epoch = findLastCheckpoint(save_dir=save_dir)  # load the last model in matconvnet style
    for e in range(1, initial_epoch+1):
        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(clean_dir)):
            batch_x = io.imread(clean_dir[i])
            noise_name = clean_dir[i].split('/')[3].split('.')[0][:-4] + 'real.png'
            noise_path = os.path.join(test_dir, noise_name)
            batch_y = io.imread(noise_path)

            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)

            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) / 255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) / 255.0

            psnr_list += [compare_psnr(out, batch_x)]
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
#             out_name = noise_name[:-8] + 'PSNR_' + '%.2f'%psnr_list[-1] + '_SSIM_' + '%.4f'%ssim_list[-1] + '.png'
#             io.imsave(os.path.join(result_dir, out_name), out)
#             print('%s----%.2f----%.4f'%(noise_name, np.array(psnr_list[-1]), np.array(ssim_list[-1])))

        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()
            
            
        print('Epoch: %d, PSNR: %.3f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Epoch: 1, PSNR: 35.474, SSIM: 0.9650
Epoch: 2, PSNR: 36.047, SSIM: 0.9696
Epoch: 3, PSNR: 36.010, SSIM: 0.9693
Epoch: 4, PSNR: 36.015, SSIM: 0.9694
Epoch: 5, PSNR: 36.016, SSIM: 0.9693
Epoch: 6, PSNR: 36.058, SSIM: 0.9697
Epoch: 7, PSNR: 36.027, SSIM: 0.9695
Epoch: 8, PSNR: 36.019, SSIM: 0.9694
Epoch: 9, PSNR: 36.075, SSIM: 0.9699
Epoch: 10, PSNR: 36.023, SSIM: 0.9694
Epoch: 11, PSNR: 36.017, SSIM: 0.9692
Epoch: 12, PSNR: 36.363, SSIM: 0.9713
Epoch: 13, PSNR: 36.417, SSIM: 0.9717
Epoch: 14, PSNR: 36.405, SSIM: 0.9716
Epoch: 15, PSNR: 36.344, SSIM: 0.9713
Epoch: 16, PSNR: 36.441, SSIM: 0.9720
Epoch: 17, PSNR: 36.399, SSIM: 0.9718
Epoch: 18, PSNR: 36.417, SSIM: 0.9717
Epoch: 19, PSNR: 36.440, SSIM: 0.9722
Epoch: 20, PSNR: 36.408, SSIM: 0.9721
Epoch: 21, PSNR: 36.446, SSIM: 0.9721
Epoch: 22, PSNR: 36.373, SSIM: 0.9714
Epoch: 23, PSNR: 36.433, SSIM: 0.9720
Epoch: 24, PSNR: 36.452, SSIM: 0.9722
Epoch: 25, PSNR: 36.430, SSIM: 0.9720
Epoch: 26, PSNR: 36.417, SSIM: 0.9719
Epoch: 27, PSNR: 36.4